## JUSTICE MATTER stepwise run evaluation

In [13]:
import numpy as np
import os
import matplotlib.pyplot as plt

def load_npz_file(npz_file_path):
    """
    Load a .npz file and return the relevant variables.
    """
    data = np.load(npz_file_path, allow_pickle=True)
    array = data[data.files[0]].item()  # Assuming the file contains a single dictionary
    return array

def plot_scenario(data, scenario_name, output_dir=None):
    """
    Plot the specified variables for a given scenario.
    """
    variables_of_interest = ['depletion_ratio', 'recycling_cost', 'emissions_avoided', 'emissions']
    titles = ['Depletion Ratio', 'Recycling Cost (Trillions)', 'Emissions Avoided (Gt)', 'Emissions (Gt)']
    
    # Create a figure with 2 rows and 2 columns
    fig, axs = plt.subplots(2, 2, figsize=(12, 10))
    fig.suptitle(f'CE Global projections: {scenario_name}', fontsize=16)
    
    years = np.arange(2015, 2301)
    
    for i, var in enumerate(variables_of_interest):
        variable_data = data[var]
        mean_data = np.mean(variable_data, axis=2)  # Mean across regions

        ax = axs[i//2, i%2]
        for j in range(mean_data.shape[0]):  # Iterate over ensembles
            ax.plot(years, mean_data[j,:])  
            
        ax.set_title(titles[i])
        ax.set_xlabel('Year')
        ax.set_ylabel('Value')
    
    plt.tight_layout(rect=[0, 0, 1, 0.96])
    
    # Save the plot
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    plt.savefig(os.path.join(output_dir, f'{scenario_name}.png'))
    plt.close()

def process_all_scenarios(input_dir, output_dir='plots'):
    """
    Process all .npz files in the specified directory.
    """
    for file_name in os.listdir(input_dir):
        if file_name.endswith('.npz'):
            npz_file_path = os.path.join(input_dir, file_name)
            scenario_name = os.path.splitext(file_name)[0]
            data = load_npz_file(npz_file_path)
            plot_scenario(data, scenario_name, output_dir)

# Example usage:
input_dir = 'data/output/ce'
output_dir = 'figures/ce'
process_all_scenarios(input_dir, output_dir)
